Install sympy library (if required)

`pip install sympy`

SymPy is Python's library for symbolic mathematics.

In [8]:
import sympy as s

We can create an arbitrary expression like so:

In [9]:
x = s.Symbol("x")
y = s.Symbol("y")
expr = x*y + 3

For now we are interested in SymPy's expression trees (https://docs.sympy.org/latest/tutorials/intro-tutorial/manipulation.html) - these allow us to draw a computation graph for an arbitrary expression.

SymPy's `srepr` function can print us such a graph:

In [10]:
s.srepr(expr)

"Add(Mul(Symbol('x'), Symbol('y')), Integer(3))"

Note that we basically have a deeply-nested tree structure here.

We can recurse through this tree: https://docs.sympy.org/latest/tutorials/intro-tutorial/manipulation.html#recursing-through-an-expression-tree.

**Exercise**: use an example in the "Walking the Tree" section from above link to implement Python code that will:
1. Recurse through an arbitrary expression tree and collect (parent, child) pairs (`(func, args[i])` pairs).
2. Print out a list of nodes in Tikz format, for example:
   ```
     a -> b;
     b -> {c, d, e};
     a -> f;
   ```
   where instead of `a`, `b`, and so on, there will be math expressions.
3. Verify that the graph is displayed by using the template here: https://www.overleaf.com/project/6613f56ba03909e4a99556a8

In [12]:
import sympy as sp

def walk_tree(expr):
    """ Recursively walk through the expression tree and collect (parent, child) pairs. """
    if expr.args:
        for arg in expr.args:
            print(f"{expr} -> {arg};")
            walk_tree(arg)

x, y, z = sp.symbols('x y z')
expr = sp.sympify('x + (y * z)')

walk_tree(expr)


x + y*z -> x;
x + y*z -> y*z;
y*z -> y;
y*z -> z;


Код для LaTeX

\documentclass{standalone}
\usepackage{tikz}

\begin{document}
\begin{tikzpicture}[
  node distance={15mm}, 
  main/.style = {draw, circle}]

\node[main] (1) {x + y*z};
\node[main] (2) [below left of=1] {x};
\node[main] (3) [below right of=1] {y*z};
\node[main] (4) [below left of=3] {y};
\node[main] (5) [below right of=3] {z};

\draw[->] (1) -- (2);
\draw[->] (1) -- (3);
\draw[->] (3) -- (4);
\draw[->] (3) -- (5);

\end{tikzpicture}
\end{document}


### Notes on Tikz tree drawing:
1. Please switch LaTeX engine to LuaTeX: https://www.overleaf.com/learn/how-to/Changing_compiler
2. Note that parent->child relationships are created like so:
   ```
   parent -> child;
   ```
   or
   ```
   parent -> {child1, child2, ...}
   ```
   for multiple children
3. Same nodes can be repeated on multiple lines - Tikz's algorithm will take care of that.
4. Custom labels can be set via `b/"Custom label"` syntax. This custom label can be set only once when the node is first mentioned.
5. More info here: https://tikz.dev/gd-trees

In [13]:
def walk_tree(expr):
    """Recursively walk through the expression tree and collect (parent, child) pairs."""
    nodes = set()
    edges = []
    
    def recurse(expr, parent=None):
        nodes.add(expr)
        if parent is not None:
            edges.append((parent, expr))
        for arg in expr.args:
            recurse(arg, expr)
    
    recurse(expr)
    
    return nodes, edges

x, y, z = sp.symbols('x y z')
expr = sp.sympify('x + y * z')

nodes, edges = walk_tree(expr)

for parent, child in edges:
    print(f'"{parent}" -> "{child}";')

for i, node in enumerate(nodes):
    print(f'"{node}" [label="Node {i}"];')


"x + y*z" -> "x";
"x + y*z" -> "y*z";
"y*z" -> "y";
"y*z" -> "z";
"x" [label="Node 0"];
"y*z" [label="Node 1"];
"y" [label="Node 2"];
"x + y*z" [label="Node 3"];
"z" [label="Node 4"];


\documentclass{standalone}
\usepackage{tikz}
\usetikzlibrary{arrows.meta, positioning}

\begin{document}
\begin{tikzpicture}[
    >={Stealth[round]},
    every node/.style={draw, minimum size=1cm},
    level 1/.style={sibling distance=5cm, level distance=1.5cm},
    level 2/.style={sibling distance=2.5cm, level distance=1.5cm}]
    
    \node {x + y*z}
        child { node {x} }
        child { node {y*z}
            child { node {y} }
            child { node {z} }
        };
\end{tikzpicture}
\end{document}
